In [36]:
#!pip install --upgrade matplotlib
#!pip install graphviz
#!pip install --upgrade pip
#!pip uninstall matplotlib
#!pip install --user matplotlib

In [109]:
import pandas as pd
import numpy as np
import matplotlib as plt
import graphviz
import warnings
warnings.filterwarnings('ignore')

In [110]:
path_to_file = 'dataset.pkl'
df = pd.read_pickle(path_to_file)
print(df.columns)

Index(['actq', 'apq', 'atq', 'ceqq', 'cheq', 'cogsq', 'csh12q', 'cshfdq',
       'cshiq', 'cshopq',
       ...
       'sector_code_815.0', 'sector_code_817.0', 'sector_code_822.0',
       'sector_code_823.0', 'sector_code_825.0', 'sector_code_830.0',
       'sector_code_835.0', 'sector_code_840.0', 'sector_code_845.0',
       'sector_code_850.0'],
      dtype='object', length=731)


In [111]:
df = df.drop([x  for x in df.columns if 'fqtr' in x], axis=1)
data = df.replace([np.inf,-np.inf],np.nan) # change inf, -inf to nan
data.reset_index(inplace=True)
#data = data.groupby('ticker').fillna(method='ffill') # forward-fill propagates the last observed non-null value forward until another non-null value is encountered.
data = data.fillna(method='ffill')
data = data.fillna(0) # fill all other nans to 0
data

,date,ticker,actq,apq,atq,ceqq,cheq,cogsq,csh12q,cshfdq,...,sector_code_815.0,sector_code_817.0,sector_code_822.0,sector_code_823.0,sector_code_825.0,sector_code_830.0,sector_code_835.0,sector_code_840.0,sector_code_845.0,sector_code_850.0
0,2000-02-01,LLB,3.540,0.143,7.668,6.732,2.553,0.458,6.3910,6.391,...,0,0,0,0,0,0,0,0,0,0
1,2000-02-03,MYR,107.661,24.387,220.463,136.555,1.049,36.883,25.5360,22.082,...,0,0,0,0,0,0,0,0,0,0
2,2000-02-08,LZB,447.719,57.893,740.905,460.612,16.531,274.525,52.2660,52.274,...,0,0,0,0,0,0,0,0,0,0
3,2000-02-08,SJM,234.415,33.821,488.136,322.432,26.054,91.172,28.8808,28.603,...,0,0,0,0,0,0,0,0,0,0
4,2000-02-09,CSCO,7722.000,482.000,21391.000,16523.000,3968.000,1422.000,3374.1250,3648.000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294407,2018-12-31,ELLO,77.786,1.907,255.123,90.949,61.530,2.272,10.9220,10.676,...,0,0,0,0,0,0,0,0,0,0
294408,2018-12-31,MFCB,84.040,20.586,252.126,186.153,59.330,76.382,12.5350,12.535,...,0,0,1,0,0,0,0,0,0,0
294409,2018-12-31,MFCB,79.628,16.437,242.757,193.929,56.640,4.650,12.5350,12.535,...,0,0,1,0,0,0,0,0,0,0
294410,2018-12-31,PETZ,13.670,6.395,18.833,6.459,2.524,12.253,9.3245,9.424,...,0,0,0,0,0,0,0,0,0,0


In [112]:
def f(x):
    if x > 0.05:
        return 1
    elif x <= -0.01:
        return -1
    else:
        return 0
    
data['rel_performance'] = data['pred_rel_return'].apply(f)
data['rel_performance']

0        -1
1         1
2         1
3         1
4        -1
         ..
294407    0
294408    0
294409    0
294410    0
294411    0
Name: rel_performance, Length: 294412, dtype: int64

## Create both a DecisionTree classifier and a BaggingClassifier

In [113]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

In [114]:
data = data[data['market_cap'] > 1000.0]
print(data.shape)
#data.reset_index(inplace=True)
data.set_index('date',inplace=True)
data.head()

(111468, 730)


,ticker,actq,apq,atq,ceqq,cheq,cogsq,csh12q,cshfdq,cshiq,...,sector_code_817.0,sector_code_822.0,sector_code_823.0,sector_code_825.0,sector_code_830.0,sector_code_835.0,sector_code_840.0,sector_code_845.0,sector_code_850.0,rel_performance
date,,,,,,,,,,,,,,,,,,,,,
2000-02-09,CSCO,7722.000,482.000,21391.000,16523.000,3968.000,1422.000,3374.1250,3648.000,3445.000,...,0,0,0,0,0,0,0,0,0,-1
2000-02-09,ROP,172.725,19.662,474.649,239.432,3.198,47.634,30.2688,31.217,3445.000,...,0,0,0,0,0,0,0,0,0,1
2000-02-10,CMOS,240.767,27.044,376.536,209.411,68.625,43.023,21.4360,23.753,3445.000,...,0,0,0,0,0,0,0,0,0,1
2000-02-11,DELL,7681.000,3538.000,11471.000,5308.000,4132.000,5452.000,2536.0000,2731.000,2575.000,...,0,0,0,0,0,0,0,0,0,1
2000-02-15,VAL,507.082,139.497,1094.080,402.382,27.605,221.366,43.1858,43.687,53.321,...,0,0,0,0,0,0,0,0,0,0


### Create train, validation and test datasets

In [115]:
df_train = data.loc['2007':'2009']
df_valid = data.loc['2010-04-01':'2010-07-01']
df_test = data.loc['2010':'2018']

In [123]:
columns_to_delete = ['ticker',
                     'date',
                     'next_period_return',
                     'spy_next_period_return',
                     'rel_performance',
                     'pred_rel_return',
                     'return', 
                     'cum_ret', 
                     'spy_cum_ret']

train = df_train.reset_index().drop(columns_to_delete, axis=1)

valid = df_valid.reset_index().drop(columns_to_delete, axis=1)

test = df_test.reset_index().drop(columns_to_delete, axis=1)

In [124]:
train_stock_returns = df_train['next_period_return']
valid_stock_returns = df_valid['next_period_return']
test_stock_returns = df_test['next_period_return']

### The targets for train, test, val

In [125]:
y_train = df_train['rel_performance'].values
y_valid = df_valid['rel_performance'].values
y_test = df_test['rel_performance'].values

### Scale the features

In [130]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
float_vars = [x for x in train.columns if data[x].dtype == 'float64']
int_vars = [x for x in train.columns if data[x].dtype == 'unit8']
vrs = [x for x in train.columns if (data[x].dtype == 'unit8' or data[x].dtype == 'float64')]

#train[float_vars] = scaler.fit_transform(train[float_vars])
#valid[float_vars] = scaler.transform(valid[float_vars])
#test[float_vars] = scaler.transform(test[float_vars])
train = scaler.fit_transform(train)
valid = scaler.transform(valid)
test = scaler.transform(test)

In [132]:
test

array([[ 0.24541996, -0.12117293, -0.11187534, ..., -0.07111792,
        -0.0343775 , -0.24178486],
       [-0.35329216, -0.12650906, -0.17688577, ..., -0.07111792,
        -0.0343775 , -0.24178486],
       [ 0.02420756, -0.11784572, -0.15625824, ..., -0.07111792,
        -0.0343775 , -0.24178486],
       ...,
       [-0.25848371, -0.12516176, -0.17251651, ..., -0.07111792,
        -0.0343775 , -0.24178486],
       [ 2.2237177 ,  0.09980031,  0.21515862, ..., -0.07111792,
        -0.0343775 , -0.24178486],
       [-0.01104469, -0.12035293, -0.07931036, ..., -0.07111792,
        -0.0343775 , -0.24178486]])

## Train the models

In [133]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [134]:
t_clf = DecisionTreeClassifier(min_samples_leaf = 600, max_depth = 8, random_state = 123)
t_clf.fit(train, y_train)
print(accuracy_score(y_train, t_clf.predict(train)))
print(accuracy_score(y_valid, t_clf.predict(valid)))
print(classification_report(y_train, t_clf.predict(train)))
print(classification_report(y_valid, t_clf.predict(valid)))

0.48111332007952284
0.39000693962526023
              precision    recall  f1-score   support

          -1       0.47      0.74      0.57      6872
           0       0.00      0.00      0.00      2699
           1       0.50      0.41      0.45      6525

    accuracy                           0.48     16096
   macro avg       0.32      0.38      0.34     16096
weighted avg       0.40      0.48      0.43     16096

              precision    recall  f1-score   support

          -1       0.40      0.66      0.50       595
           0       0.00      0.00      0.00       298
           1       0.37      0.31      0.34       548

    accuracy                           0.39      1441
   macro avg       0.26      0.32      0.28      1441
weighted avg       0.31      0.39      0.33      1441



In [135]:
import graphviz
from sklearn import tree

#### Хоча вибило помилку, але дерево згенерувало у файлик tree_1

In [136]:
file = tree.export_graphviz(t_clf)
graph = graphviz.Source(file)
graph.render('tree_1')

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [137]:
import optuna
from optuna.trial import Trial
from functools import partial

In [138]:
def objective(trial:Trial,
              train=None,
              labels=None,
              val=None,
              val_labels=None,
              val_rets=None):

    t_min_samples_leaf = trial.suggest_int('min_samples_leaf', 100, 1200, step=100)
    t_max_depth = trial.suggest_int('max_depth', 5, 25, step=5)

    
    t_clf = DecisionTreeClassifier(min_samples_leaf = t_min_samples_leaf, 
                                   max_depth = t_max_depth, 
                                   random_state = 123)
    t_clf.fit(train, labels)

    preds = t_clf.predict(val)
    profit = (preds * val_rets).sum()

    return profit

In [139]:
study = optuna.create_study(direction="maximize")

[I 2023-12-20 22:52:53,336] A new study created in memory with name: no-name-c8a85035-25a0-4426-8fbe-b5271b38f1d3


In [142]:
%%time
study.optimize(partial(objective, train=train, labels=y_train, val=valid, val_labels=y_valid, val_rets=valid_stock_returns.values), n_trials=50, n_jobs=-1)

[I 2023-12-20 22:57:47,777] Trial 20 finished with value: 14.994006000000045 and parameters: {'min_samples_leaf': 300, 'max_depth': 10}. Best is trial 15 with value: 21.63547399999997.
[I 2023-12-20 22:57:47,894] Trial 31 finished with value: 5.1456179999999865 and parameters: {'min_samples_leaf': 400, 'max_depth': 10}. Best is trial 15 with value: 21.63547399999997.
[I 2023-12-20 22:57:47,925] Trial 26 finished with value: 14.994006000000045 and parameters: {'min_samples_leaf': 300, 'max_depth': 10}. Best is trial 15 with value: 21.63547399999997.
[I 2023-12-20 22:57:47,959] Trial 21 finished with value: 14.994006000000045 and parameters: {'min_samples_leaf': 300, 'max_depth': 10}. Best is trial 15 with value: 21.63547399999997.
[I 2023-12-20 22:57:47,992] Trial 27 finished with value: 14.994006000000045 and parameters: {'min_samples_leaf': 300, 'max_depth': 10}. Best is trial 15 with value: 21.63547399999997.
[I 2023-12-20 22:57:48,033] Trial 22 finished with value: 14.99400600000004

Wall time: 7.27 s


In [143]:
study.best_params

{'min_samples_leaf': 200, 'max_depth': 5}

### Instantiating the DecisionTreeClassifier with hyper-parameters and train the model

In [147]:
t_clf = DecisionTreeClassifier(**study.best_params, random_state=123)
t_clf.fit(train,y_train)
print(accuracy_score(y_train, t_clf.predict(train)))
print(accuracy_score(y_valid, t_clf.predict(valid)))

0.47775844930417494
0.4066620402498265
